TensorFlow 共享变量以及分配

In [1]:
import numpy as np
import tensorflow as tf

config = tf.ConfigProto(log_device_placement=True)
sess = tf.Session(config=config)

In [2]:
with tf.name_scope('name_scope_x'):
    var1 = tf.get_variable(name='var1', shape=[1], dtype=tf.float32)
    var2 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)
    var3 = tf.get_variable(name='var2', shape=[2], dtype=tf.float32)
    var4 = tf.Variable(name='var2', initial_value=[2], dtype=tf.float32)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(var1.name, sess.run(var1))
    print(var2.name, sess.run(var2))
    print(var3.name, sess.run(var3))
    print(var4.name, sess.run(var4))

var1:0 [-0.84900182]
name_scope_x/var2:0 [ 2.]
var2:0 [-1.18312562  0.46193779]
name_scope_x/var2_1:0 [ 2.]
